In [ ]:
%pip install mysql.connector
%pip install xlrd
%pip install pandas
%pip install python-dotenv

In [2]:
import xlrd
import mysql.connector
import pandas as pd
import os

In [ ]:
# Load environment variables
from dotenv import load_dotenv
load_dotenv(verbose=True)

In [8]:
# Connect to the MySQL database using the created environment variables
conexion = mysql.connector.connect(
    host=os.getenv("EV_host"),
    user=os.getenv("EV_user"),
    password=os.getenv("EV_password"),
    database=os.getenv("EV_database"),
)

In [9]:
cursor = conexion.cursor()                      # We install a cursor to execute SQL queries

nombre_base_datos = "sandbox"                   # We define the database and tables as variables
nombre_tabla_ventas = "raw_ventas"
nombre_tabla_calendario = "raw_calendario"
nombre_tabla_pedidos = "raw_pedidos"

cursor.execute(f"USE {nombre_base_datos}")

### We delete any existing tables, and create the table of each type that we will use in this project, selecting the data type for each field:

In [10]:

cursor.execute(f"DROP TABLE IF EXISTS {nombre_tabla_ventas}")
cursor.execute(f"CREATE TABLE IF NOT EXISTS {nombre_tabla_ventas} ("
               "FAMILIA VARCHAR(255),"
               "Tipo VARCHAR(255),"
               "FechaVenta DATE,"
               "HoraVenta INT,"
               "Articulo VARCHAR(255),"
               "Cantidad FLOAT,"
               "Precio FLOAT,"
               "Importe FLOAT"
               ")")

In [12]:
cursor.execute(f"DROP TABLE IF EXISTS {nombre_tabla_calendario}")
cursor.execute(f"CREATE TABLE IF NOT EXISTS {nombre_tabla_calendario} ("
               "Fecha DATE,"
               "Festivo VARCHAR(255)"
               ")")

In [11]:
cursor.execute(f"DROP TABLE IF EXISTS {nombre_tabla_pedidos}")
cursor.execute(f"CREATE TABLE IF NOT EXISTS {nombre_tabla_pedidos} ("
               "Tipo VARCHAR(255),"
               "Fecha DATE,"
               "Articulo VARCHAR(255),"
               "Cantidad FLOAT,"
               "Precio FLOAT,"
               "Importe FLOAT"
               ")")

In [ ]:
# To work with .xlsx files in the environment
%pip install openpyxl

### Load data and read

In [ ]:
# Load xlsx file
nombre_archivo_ventas = "data/ArticulosPanaderia.xlsx"

# Open xlsx file
df = pd.read_excel(nombre_archivo_ventas)

# Prepare the data for batch insertion
registros = []
for _, row in df.iterrows():
    familia = row['FAMILIA']
    tipo = row['Tipo']
    fecha_venta = row['FechaVenta'].strftime('%Y-%m-%d')
    hora_venta = row['HoraVenta']
    articulo = row['Articulo']
    cantidad = row['Cantidad']
    precio = row['Precio']
    importe = row['Importe']
    registros.append((familia, tipo, fecha_venta, hora_venta, articulo, cantidad, precio, importe))

# Select database
cursor.execute("USE sandbox")

# Create the SQL query to insert the data into the table
consulta = f"INSERT INTO {nombre_tabla_ventas} (FAMILIA, Tipo, FechaVenta, HoraVenta, Articulo, Cantidad, Precio, Importe) " \
           "VALUES (%s, %s, %s, %s, %s, %s, %s, %s)"

# Execute the query with the batch list of records
tamano_lote = 1000
for i in range(0, len(registros), tamano_lote):
    lote_registros = registros[i:i + tamano_lote]
    cursor = conexion.cursor()
    cursor.executemany(consulta, lote_registros)
    conexion.commit()

In [ ]:
nombre_archivo_calendario = "data/Calendario.xlsx"

df = pd.read_excel(nombre_archivo_calendario)

registros = []
for _, row in df.iterrows():
    fecha = row['Fecha'].strftime('%Y-%m-%d')
    festivo = row['Festivo']
    registros.append((fecha,festivo))

consulta = f"INSERT INTO {nombre_tabla_calendario} (Fecha, Festivo) " \
           "VALUES (%s, %s)"

cursor.execute("USE sandbox")

tamano_lote = 1000
for i in range(0, len(registros), tamano_lote):
    lote_registros = registros[i:i + tamano_lote]
    cursor = conexion.cursor()
    cursor.executemany(consulta, lote_registros)
    conexion.commit()

In [ ]:
nombre_archivo_pedidos = "data/CantidadPedida.xlsx"

df = pd.read_excel(nombre_archivo_pedidos)

registros = []
for _, row in df.iterrows():
    tipo = row['Tipo']
    fecha_venta = row['Fecha'].strftime('%Y-%m-%d')
    articulo = row['Articulo']
    cantidad = row['Cantidad']
    precio = row['Precio']
    importe = row['Importe']
    registros.append(( tipo, fecha_venta, articulo, cantidad, precio, importe))

cursor.execute("USE sandbox")

consulta = f"INSERT INTO {nombre_tabla_pedidos} (Tipo, Fecha, Articulo, Cantidad, Precio, Importe) " \
           "VALUES (%s, %s, %s, %s, %s, %s)"

tamano_lote = 1000
for i in range(0, len(registros), tamano_lote):
    lote_registros = registros[i:i + tamano_lote]
    cursor = conexion.cursor()
    cursor.executemany(consulta, lote_registros)
    conexion.commit()

In [15]:
# Save a series of queries in a variable to execute later

consulta_proceso="""


/*Ordenamos los productos por familia*/
drop table if exists sandbox.articulos_top;
create table sandbox.articulos_top as (

	select
	Articulo,
	FAMILIA,
	sum(importe) as importe_total,
	ROW_NUMBER () OVER(partition by FAMILIA order by sum(importe) desc) as orden
	from sandbox.raw_ventas
	where FechaVenta >= '2021-05-01'
	group by 1,2

);


/*Creamos un calendario general del 1/1/2017 al 31/12/2023*/
drop table if exists sandbox.calendario_dias;
create table sandbox.calendario_dias as (

with recursive cte_calendario as (
	select date('2017-01-01') as calendar_date -- fecha de inicio
	union all
	select date_add(calendar_date, interval 1 day) as calendar_date from cte_calendario
	where date_add(calendar_date, interval 1 day) <= date('2023-12-31') -- condicion de fin, fecha final
)

select
calendar_date as fecha,
year(calendar_date) as fx_anno,
month(calendar_date) as fx_mes,
day(calendar_date) as fx_day,
date_format(calendar_date, '%Y%m') as fx_anno_mes,
date_format(calendar_Date,'%x-%v') as semana -- formato
from cte_calendario

);

/*Añadimos al calendario los festivos*/
drop table if exists sandbox.calendario_completo;
create table sandbox.calendario_completo as (
select
base.*,
festivos.festivo
from sandbox.calendario_dias base
left join
	(select
	a.*,
	row_number() over(partition by a.fecha order by a.festivo) as orden
	from sandbox.raw_calendario a
	) festivos
on base.fecha=festivos.fecha
and festivos.orden=1
);

/*Agregamos las ventas por día (quitamos el detalle por hora)*/

/*Añadimos además el orden del producto y si está en el rango de fecha de estudio*/
drop table if exists sandbox.ventas_diarias;
create table sandbox.ventas_diarias as (
select
base.familia,
base.tipo,
base.fechaVenta,
calendario.festivo,
base.articulo,
sum(base.precio*base.cantidad)/sum(base.cantidad) as precio, /* Se calcula como el precio promedio del día, ya que hay cambios de precio a lo largo del día*/
articulos.orden as orden_articulo_familia,
case when base.fechaVenta >=date('2021-05-01') then 'S' else 'N' end as in_fecha_estudio,
sum(base.cantidad) as cantidad,
sum(base.importe) as importe

from sandbox.raw_ventas base

inner join sandbox.calendario_completo calendario
on base.FechaVenta=calendario.fecha

inner join sandbox.articulos_top articulos
on base.familia=articulos.familia
and base.articulo=articulos.articulo


group by 1,2,3,4,5,7,8);


/*Creamos una vista final con el filtrado para el estudio*/
drop view if exists sandbox.ventas_diarias_estudio_completo;
create view sandbox.ventas_diarias_estudio_completo as
(select *
from sandbox.ventas_diarias
where tipo='VENTA'
and in_fecha_estudio='S'
and orden_articulo_familia<=5);


/*Creamos una vista extra con el filtrado para el estudio que no tenga días de mayo 23 (que habrá que predecir)*/
drop view if exists sandbox.ventas_diarias_estudio;
create view sandbox.ventas_diarias_estudio as
(select *
from sandbox.ventas_diarias
where tipo='VENTA'
and in_fecha_estudio='S'
and fechaventa < date('2023-05-01')
and orden_articulo_familia<=5);
"""

In [16]:
# Separate the queries into a list
consultas_lista = consulta_proceso.split(";")

# Remove whitespace and empty elements
consultas_lista = [consulta.strip() for consulta in consultas_lista if consulta.strip()]

# Execute the queries
for consulta in consultas_lista:
    cursor.execute(consulta)

# Confirm changes
conexion.commit()